**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [3]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

  Using cached google_colab_selenium-1.0.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached selenium-4.32.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
Using cached google_colab_selenium-1.0.14-py3-none-any.whl (8.2 kB)
Using cached selenium-4.32.0-py3-none-any.whl (9.4 MB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# 演習用のコンテンツを取得
!git clone https://github.com/YushinOgasawara/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 133 (delta 17), reused 18 (delta 15), pack-reused 103 (from 3)
Receiving objects: 100% (133/133), 1.36 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [5]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
import random
random.seed(0)

In [8]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [9]:
Q1 = "大規模言語モデルとは何ですか？"
Q2 = "ファインチューニングとは何ですか？"
Q3 = "大規模言語モデルにおけるスケーリング則とは何ですか？"
Q4 = "スケール則はどのようにモデルの性能に影響を与えますか？"
Q5 = "大規模言語モデルにおけるトークナイザの機能を教えてください。"

In [10]:
messages = [
    {"role": "user", "content": Q1},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [11]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

 大規模言語モデル (LLM) は、膨大な量のテキストデータから学習された、人間のような自然言語を理解し生成する人工知能です。 

**簡単に言うと、** 

* **大量のテキストデータから学習:**  インターネット上の文章、書籍、記事など、様々なテキストデータから学習します。
* **自然言語を理解・生成:**  学習したデータから、人間が理解できる自然な文章を生成したり、質問に答えたり、文章を要約したりすることができます。
* **多様なタスクに対応:**  文章の翻訳、要約、質問応答、文章生成など、様々なタスクに対応できます。

**特徴:**

* **複雑な文脈理解:**  LLM は、文章の全体的な意味や文脈を理解し、人間のように複雑な文章を生成できます。
* **多様なタスクに対応:**  LLM は、様々なタスクを効率的に実行できます。
* **人間と同様の会話:**  LLM は、人間と自然な会話をするように設計されています。

**例:**

* **ChatGPT:**  ユーザーの質問に回答したり、文章を生成したりする。
* **Bard:**  Googleが開発した、LLM をベースにした検索エンジン。
* **LaMDA:**  Googleが開発した、人間と自然な会話をするように設計されたLLM。



 





In [12]:
messages = [
    {"role": "user", "content": Q2},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [13]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

**ファインチューニング**とは、**既存のモデルをさらに精度や性能を向上させるためのプロセス**です。 

**従来のモデルの学習から、より詳細なデータセットを用いて、モデルの精度を向上させる**ことを指します。

**具体的には、以下の点が挙げられます:**

* **データの調整:**  既存のデータセットをより詳細なものに調整したり、新しいデータセットを追加することで、モデルの学習に役立てる
* **パラメータの調整:**  モデルの内部パラメータを微調整することで、より正確な予測を行う
* **モデルの構造の変更:**  モデルの構造を調整することで、より複雑なパターンを学習できる

**ファインチューニングのメリット:**

* **精度向上:**  既存のモデルの精度を大幅に向上させる
* **汎用性向上:**  より多くのデータセットで高い精度を維持できる
* **効率性向上:**  既存のモデルを再学習するよりも、より短時間でモデルの性能を向上させる

**ファインチューニングの例:**

* **画像認識:**  既存の画像認識モデルをより詳細な画像データセットを用いて微調整することで、より正確な画像分類を行う
* **自然言語処理:**  既存の自然言語処理モデルをより詳細なテキストデータセットを用いて微調整することで、より正確な文章生成や翻訳を行う



 





In [14]:
messages = [
    {"role": "user", "content": Q3},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [15]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

 大規模言語モデルにおけるスケーリング則とは、**モデルの規模が大きくなるにつれて、その性能が著しく向上する法則**です。 

具体的には、

* **より多くのデータ:**  より多くのデータで学習することで、モデルはより複雑な文脈を理解し、より正確な回答を生成できるようになります。
* **より多くの計算資源:**  より多くの計算資源（CPU、GPUなど）を用いることで、モデルはより複雑な計算を高速に行え、より高い精度で処理できます。
* **より複雑なモデル構造:**  より複雑なモデル構造（例えば、より多くの層やニューラルネットワーク）を用いることで、モデルはより複雑な文脈を理解し、より正確な回答を生成できます。

 

**スケーリング則のメリット**

* **より高い精度:**  より多くのデータや計算資源を用いることで、モデルの精度が向上します。
* **より複雑なタスクに対応:**  より複雑なタスクに対応できるようになります。
* **より多様な応用:**  より多様な応用分野で利用できるようになります。

 

**スケーリング則の課題**

* **コスト増加:**  スケーリング則に伴い、モデルの構築とトレーニングに必要なコストが大幅に増加します。
* **データの収集・処理:**  より多くのデータが必要になるため、データ収集・処理の効率性が求められます。
* **モデルの可視化・解釈:**  より複雑なモデル構造を持つと、モデルの可視化・解釈が難しくなる場合があります。



 





In [16]:
messages = [
    {"role": "user", "content": Q4},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [17]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

スケール則は、モデルの性能に大きく影響を与える要素です。 

**スケール則とは、データセットの規模がモデルの性能に与える影響を指します。** 

具体的には、スケール則は以下のような要素に影響を与えます。

**1. 学習速度と精度:**

* **大きなデータセット:** モデルはより多くのデータで学習できるため、精度が高まります。
* **小さなデータセット:** モデルは学習に時間がかかるだけでなく、精度が低くなる傾向があります。

**2. モデルの複雑さ:**

* **大きなデータセット:** モデルは複雑な構造を持つことが可能になり、より高い精度と複雑な表現能力を実現できます。
* **小さなデータセット:** モデルは複雑な構造を持つことが難しく、精度が低下する傾向があります。

**3. モデルの汎用性:**

* **大きなデータセット:** モデルは、新しいデータに対して高い汎用性を示す傾向があります。
* **小さなデータセット:** モデルは、新しいデータに対して汎用性が低くなり、精度が低下する傾向があります。

**4. モデルの安定性:**

* **大きなデータセット:** モデルは、より安定した結果を得ることが可能になります。
* **小さなデータセット:** モデルは、より不安定な結果を得ることがあり、精度が不安定になる傾向があります。

**スケール則の具体的な影響例:**

* **画像認識:**  大きな画像データセットを用いることで、より高い精度で物体検出や画像分類を実現できます。
* **自然言語処理:**  膨大なテキストデータを用いることで、より高い精度で文章の要約や翻訳を実現できます。
* **音声認識:**  大きな音声データセットを用いることで、より高い精度で音声認識や音声生成を実現できます。



**スケール則を考慮するメリット:**

* **より高い精度:**  より大きなデータセットを用いることで、モデルの精度が向上します。
* **より複雑なモデル:**  より大きなデータセットを用いることで、より複雑なモデルを構築できます。
* **より高い汎用性:**  より大きなデータセットを用いることで、モデルが新しいデータに対して高い汎用性を示すようになります。



 
 





In [18]:
messages = [
    {"role": "user", "content": Q5},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [19]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

 大規模言語モデルにおけるトークナイザは、**テキストをトークン（単語や小節）に分割する機能**です。 

具体的には、以下の様な役割を果たします。

**1. モデルの入力・出力の標準化:**
   - モデルは、テキストを単語や小節に分割して処理します。
   - トークナイザは、モデルの処理を効率化し、より正確な結果を得るために必要です。

**2. モデルの可視化:**
   - トークナイザは、モデルの処理過程を可視化し、理解しやすい形で表示します。
   - これにより、モデルの動作原理を理解し、問題点や改善点を見つけることができます。

**3. モデルの拡張性:**
   - トークナイザは、モデルの拡張性を向上させるために役立ちます。
   - 新しい言語や新しいテキストフォーマットに対応できるように、モデルの構造を柔軟に調整できます。

**4. モデルの効率性:**
   - トークナイザは、モデルの処理時間を短縮し、より高速な処理を実現します。
   - これにより、リアルタイムでの応答や大量のテキストの処理が可能になります。

**トークナイザの種類:**

- **単語トークナイザ:** 単語を分割する
- **小節トークナイザ:** 小節を分割する
- **構文トークナイザ:** 文法構造を分割する


 


 





## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [21]:
# 必要なライブラリをインストール（BeautifulSoup4）
!pip install beautifulsoup4

import os
from bs4 import BeautifulSoup
from google.colab import files

# 指定されたファイルパス
html_file_path = '/content/lecture-ai-engineering/day3/data/LLM_Wikipedia.html'
output_dir = os.path.dirname(html_file_path)  # 同階層のパスを取得

# BeautifulSoupを使ってHTMLをパース
with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

# 大規模言語モデルに関する説明を抽出（Wikipedia標準のクラス構造に基づく）
content_div = soup.find('div', {'class': 'mw-parser-output'})
if content_div:
    paragraphs = content_div.find_all('p')
    text_content = "\n".join([para.get_text() for para in paragraphs if para.get_text().strip()])
else:
    text_content = "本文が見つかりませんでした。"

# 抽出したテキストを同階層に保存
text_output_path = os.path.join(output_dir, 'RAG_Reference_Wikipedia_LLM_Cleaned.txt')
with open(text_output_path, 'w', encoding='utf-8') as text_file:
    text_file.write(text_content)

# ファイルパス確認とダウンロード
print(f'RAG用のテキストファイルが保存されました: {text_output_path}')

RAG用のテキストファイルが保存されました: /content/lecture-ai-engineering/day3/data/RAG_Reference_Wikipedia_LLM_Cleaned.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [23]:
with open("/content/lecture-ai-engineering/day3/data/RAG_Reference_Wikipedia_LLM_Cleaned.txt", "r") as f:
  raw_writedown = f.read()

In [28]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0])

ドキュメントサイズ:  245
ドキュメントの例: 
 大規模言語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千万から数十億）を持つ人工ニューラルネットワークで構成されるコンピュータ言語モデルで、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習（英語版）によって訓練が行われる[1]


In [29]:
# Retrievalの実行
question = Q5

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[77.93899536132812, 65.66787719726562, 73.70406341552734, 77.06995391845703, 75.01927947998047, 72.28759765625, 69.55094146728516, 74.94076538085938, 77.67411804199219, 73.56517028808594, 74.9063720703125, 75.7673568725586, 69.80760955810547, 68.80111694335938, 69.47728729248047, 68.2265396118164, 67.11017608642578, 70.3152847290039, 62.621952056884766, 65.6749038696289, 62.29743957519531, 65.08378601074219, 73.1639633178711, 63.67294692993164, 69.24845886230469, 67.69892883300781, 63.24818801879883, 67.89892578125, 62.49762725830078, 69.23992156982422, 68.75993347167969, 76.49894714355469, 66.01040649414062, 71.3297119140625, 65.85531616210938, 63.502681732177734, 66.7753677368164, 64.65103912353516, 69.18546295166016, 61.285423278808594, 71.7520980834961, 73.48992156982422, 69.63284301757812, 62.84008026123047, 67.72940063476562, 70.52787780761719, 63.822208404541016, 69.74393463134766, 70.43814849853516, 73.10503387451172, 68.91471099853516, 61.32128143310547, 73.56876373291016, 67

In [30]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 79.9338607788086)
トークナイザのもう一つの機能は、計算量を削減するためのテキスト圧縮である 


取得したドキュメント2: (Score: 77.93899536132812)
大規模言語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千万から数十億）を持つ人工ニューラルネットワークで構成されるコンピュータ言語モデルで、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習（英語版）によって訓練が行われる[1] 


取得したドキュメント3: (Score: 77.67411804199219)
さらに、大規模な言語モデルは、世の中に関するかなりの一般知識を示し、訓練中に大量の事実を「記憶」することができる[2] 


取得したドキュメント4: (Score: 77.65576934814453)
通常、LLMを訓練する前にトークナイザを訓練データセットの全体に適用し、その後は凍結する 


取得したドキュメント5: (Score: 77.39479064941406)
このことは、大規模な言語モデルを評価する際に、特に重要な課題となる[35] 




In [34]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] {Q5}"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [35]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

大規模言語モデルにおけるトークナイザの機能は、**計算量を削減するためのテキスト圧縮**です。


 
 
 



## 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

### 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)


<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。


<修正後>
---


講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [ ]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

In [ ]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時（Inference）に計算資源をスケーリングすることで、モデルがより賢くなり、性能が向上すること」という概念を正確に捉えるようになった

### 問題点
* 「Inference Time Scalingは、TransformerやLSTMなどのモデルにおいて、パラメータ数を増やすのではなく、推論時計算資源をスケーリングすることで、性能が向上すること...」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [ ]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇


---

# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:
  messages = [
      {"role": "user", "content": f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      # max_new_tokens=128,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  response = tokenizer.decode(response, skip_special_tokens=True)
  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)

In [ ]:
print(len(references))

上記より、上位4件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、4件でない可能性もあります）

In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
messages = [
    {"role": "user", "content": f"与えられる資料を参考にして回答すること。[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

---


# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

**注意事項**

**ここから先のセルを実行した場合、GPUメモリ不足になる可能性が高いです。**


このノートブックでは、GPUはT4を使用しています。
Colab Pro等を契約し、L4などのよりGPUメモリの大きいものを使用するか、モデルやその設定等を変更するなどの工夫が必要になります。

In [ ]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

In [ ]:
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])

messages = [
    {"role": "user", "content": f"[参考資料]\n{reference}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))